## Importing libraries and shihh

In [1]:
from ase.io import read, write
from ase.build.molecule import molecule
from ase.io import write
import matplotlib.pyplot as plt
from ase.visualize.plot import plot_atoms
import os
import math
import numpy as np
from ase import Atoms
from pymatgen.core.lattice import Lattice
from pymatgen.core import Structure
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core.structure import Structure
import subprocess as sp
import ase.geometry
#from modify_structure import *

## Creating color-to-species (and vice versa) dictionaries to convert between POSCAR and Rhino and bacK

In [3]:
# for POSCAR to Rhino
elements_to_rgb = {'Ag':[192,192,192], 'Hg':[210, 178, 221], 
                   'Te':[151, 136, 72], 'S':[255,255,51], 'Se':[57, 255, 20],
           'C':[102,0,0], 'N':[102,255,255], 'H':[255,204,204], 'O':[255,0,0], 'F':[130,197,243], 
           'origin':[255,255,255], 'point':[0,0,0]}
# for Rhino to POSCAR
rgb_to_elements = {}
for s in elements_to_rgb:
    rgb = elements_to_rgb[s]
    rgb_to_elements[f"{rgb[0]}.{rgb[1]}.{rgb[2]}"] = s
    
print(rgb_to_elements)

def angle_between_vectors(u, v):
    dot_product = sum(i*j for i, j in zip(u, v))
    norm_u = math.sqrt(sum(i**2 for i in u))
    norm_v = math.sqrt(sum(i**2 for i in v))
    cos_theta = dot_product / (norm_u * norm_v)
    angle_rad = math.acos(cos_theta)
    angle_deg = math.degrees(angle_rad)

    return angle_deg

{'192.192.192': 'Ag', '210.178.221': 'Hg', '151.136.72': 'Te', '255.255.51': 'S', '57.255.20': 'Se', '102.0.0': 'C', '102.255.255': 'N', '255.204.204': 'H', '255.0.0': 'O', '130.197.243': 'F', '255.255.255': 'origin', '0.0.0': 'point'}


## Convert POSCAR to points file for Rhino
-**poscar_to_xyz_rgb()**:
    Converts a VASP file into a .xyz readable points file for Rhino, with elements
        mapped to RGB values
- **species_to_rgb()**: helper function for mapping POSCAR species into Rhino readable RGB values

In [18]:
def poscar_to_xyz_rgb(path, filename, extension):
    '''Converts a VASP file into a .xyz readable points file for Rhino, with elements
        mapped to RGB values
        
        - path: the path to the POSCAR
        - filename: name of the file to be converted(without the extension)
        - extension: .vasp or '' if POSCAR'''
    poscar = Structure.from_file(f"{path}{filename}{extension}")
    run = True
    while run:
        if os.path.exists(f"{path}{filename}_rhino.xyz"):
            sp.run(f"rm {path}{filename}_rhino.xyz", shell=True)
            print("\nFile already exists. Rewriting now...")
            
        with open(f"{path}{filename}_rhino.xyz", 'a') as f:
            for i in poscar.as_dict()['sites']:
                colors = species_to_rgb(str(i['species'][0]['element']))
                lines = [f"{i['xyz'][0]} {i['xyz'][1]} {i['xyz'][2]} {colors[0]} {colors[1]} {colors[2]}\n"]
                f.writelines(lines)
            f.close()
            print("\nConverted POSCAR to XYZRGB points file")
            print(f"Written to: \n{path}{filename}_rhino.xyz")
            run = False
        
def species_to_rgb(species, rgb=elements_to_rgb):
    return rgb[species]

def remove_dupes(structure):
    print(f"Total: {len(structure)}")
    new_struct = ase.geometry.get_duplicate_atoms(structure, cutoff=0.2, delete=True)
    new_struct = ase.geometry.get_duplicate_atoms(structure, cutoff=0.2, delete=False)
    print(f"Total with duplicates removed: {len(new_struct)}")

    return structure

# make sure name of file has same name as folder path
mocha = "tert-butyl_dimethyl_1D"
path = f"/Users/adrianaladera/Desktop/MIT/research/mochas/ALL_STRUCTURES/3-amino_AgS-anilines/dimethyl_1D/{mocha}/"
# path = "/Users/adrianaladera/Desktop/MIT/research/rhino_structure_design/mochas/"
file = mocha
extension = ".vasp"
poscar_to_xyz_rgb(path, file, extension)


Converted POSCAR to XYZRGB points file
Written to: 
/Users/adrianaladera/Desktop/MIT/research/mochas/ALL_STRUCTURES/3-amino_AgS-anilines/dimethyl_1D/tert-butyl_dimethyl_1D/tert-butyl_dimethyl_1D_rhino.xyz


## Convert Rhino points file to POSCAR.vasp

### Save finished Rhino points file in the rhino_structure_design folder

- **xyz_rgb_to_poscar()**: Converts a Rhino .txt readable points back into an .xyz file, with RGB values
        mapped to back to elements
- **modify_lattice():**
    Updates the unit cell of the current .xyz file and saves in .vasp format
- **rgb_to_species()**:
    '''Converts rgb values of Rhino points into POSCAR readable species

In [20]:
def xyz_rgb_to_poscar(path, dest, filename, extension):
    '''Converts a Rhino .txt readable points back into an .xyz file, with RGB values
        mapped to back to elements
        
        - path: the path to .txt file
        - dest: where the final converted file will be saved
        - filename: name of the file to be converted(without the extension)
        - extension: .txt'''
    run = True
    flag = 0
    lattice_points = []
    lattice_origin = None
    while run:
        if not os.path.exists(f"{dest}{filename}_FINAL.xyz") or flag == 1:
            with open(f"{path}{filename}{extension}", 'r') as f:
                with open(f"{dest}{filename}_FINAL.xyz", 'w') as fuck:
                    lines = f.readlines()
                    # Ag, S, C, H, N = [],[],[],[],[]
                    species_dict, species_list = {},{}
                    for i in lines:
                        i = i.strip('\n')
                        species = rgb_to_species([int(i.split(',')[3]), int(i.split(',')[4]), int(i.split(',')[5])])
                        if species not in species_dict.keys() and species != "origin" and species != "point":
                            species_dict[species] = f"{int(i.split(',')[3])} {int(i.split(',')[4])} {int(i.split(',')[5])}"
                            species_list[species] = []
                    for i in lines:
                        i = i.strip('\n')
                        species = rgb_to_species([int(i.split(',')[3]), int(i.split(',')[4]), int(i.split(',')[5])])
                        if species != "origin" and species != "point":
                            species_list[species].append([float(i.split(',')[0]), float(i.split(',')[1]),float(i.split(',')[2])])
                        elif species == "point":
                            lattice_points.append([float(i.split(',')[0]), float(i.split(',')[1]),float(i.split(',')[2])])
                        elif species == "origin":
                            lattice_origin = [float(i.split(',')[0]), float(i.split(',')[1]),float(i.split(',')[2])]
                    s, s2 = '', ''
                    s += f"{sum([len(species_list[spec]) for spec in species_list])}\n"
                    for spec in species_list:
                        s += f"{spec}{len(species_list[spec])} "
                        for a in species_list[spec]:
                            s2 += f"{spec} {a[0]} {a[1]} {a[2]}\n"
                    s += '\n' + s2
                    fuck.write(s)
            print("Converting text file to XYZ readable molecule file")
            print(f"Written to {dest}{filename}_FINAL.xyz")
            run = False
            fuck.close()
        else:
            sp.run(f"rm {dest}{filename}_FINAL.xyz", shell=True)
            print("\nFile already exists. Rewriting now...")
            run = True
            flag = 1

    lattice_points.append(lattice_origin)
    print(lattice_points)
    return lattice_points

def modify_lattice(path, dest, file, lattice_points, lattice_path=None):
    '''Updates the unit cell of the current .xyz file and saves in .vasp format
    
        - path: the path to the POSCAR
        - dest: where the final converted file will be saved
        - file: name of the .xyz file to be converted
        - lattice_path: path to the POSCAR of the desired lattice matrix, 
                        leave as '' if entering a, b, c, alpha, beta, gamma'''
    if os.path.exists(f'{path}{file}'):
        old_struct = read(f'{path}{file}') 
        mocha = remove_dupes(old_struct)
        print(type(old_struct), type(mocha))
        
        if lattice_path is None:
            # x = input("\Enter a, b, c, alpha, beta, gamma: ")
            # p = [float(i) for i in x.strip().split()]
            # long_lattice = Lattice.from_parameters(p[0],p[1],p[2],p[3],p[4],p[5]) 
            matrix = np.array(lattice_points)

            new_lattice = Lattice(np.array([matrix[0] - matrix[3],
                                            matrix[1] - matrix[3],
                                            matrix[2] - matrix[3],]))
            mocha.set_cell(new_lattice.matrix)  
        else:
            lats = Structure.from_file(lattice_path)
            mocha.set_cell(lats.lattice.matrix)
        
        write(f"{dest}{file[:-4]}.vasp", mocha)
        print(f"Updating unit cell of {file} and writing to *.vasp\n")
        print(f"Written to:\n\n{dest}{file[:-4]}.vasp")
              
def rgb_to_species(rgb_vals, rgb = rgb_to_elements):
    '''Converts rgb values of Rhino points into POSCAR readable species'''
    key = f"{rgb_vals[0]}.{rgb_vals[1]}.{rgb_vals[2]}"
    return rgb[key]

# path =  "/Users/adrianaladera/Desktop/MIT/research/rhino_structure_design/mochas/"
# lattice_path = "/Users/adrianaladera/Desktop/MIT/research/MOChAs/original_structures/3-methoxy/POSCAR"
# xyz_rgb_to_poscar(path, path, 'DEMO', '.txt')
# write_path = modify_lattice(path, path, "DEMO.xyz", lattice_path)

file = "2-methoxy_dimethyl_1D"
path = "/Users/adrianaladera/Desktop/MIT/research/rhino_structure_design/mochas/"
dest = f"/Users/adrianaladera/Desktop/MIT/research/mochas/ALL_STRUCTURES/3-amino_AgS-anilines/dimethyl_1D/{file}/"
lattice_path = f"{dest}{file}.vasp" # leave as '' if entering a,b,c,alpha,beta,gamma
extension = ".txt"
lattice_points = xyz_rgb_to_poscar(path, dest, file, extension)
write_path = modify_lattice(dest, dest, f"{file}_FINAL.xyz", lattice_points)



File already exists. Rewriting now...
Converting text file to XYZ readable molecule file
Written to /Users/adrianaladera/Desktop/MIT/research/mochas/ALL_STRUCTURES/3-amino_AgS-anilines/dimethyl_1D/2-methoxy_dimethyl_1D/2-methoxy_dimethyl_1D_FINAL.xyz
[[-2.668638107417006, -4.442564872901379, -3.460172074600912], [11.36642156862756, 13.5180717070553, -3.409292084289355], [11.36642156862756, 5.977813328692804, 35.17178970202328], [11.36642156862756, -4.442564872901379, -3.460172074600912]]
Total: 168
Total with duplicates removed: 0
<class 'ase.atoms.Atoms'> <class 'ase.atoms.Atoms'>
Updating unit cell of 2-methoxy_dimethyl_1D_FINAL.xyz and writing to *.vasp

Written to:

/Users/adrianaladera/Desktop/MIT/research/mochas/ALL_STRUCTURES/3-amino_AgS-anilines/dimethyl_1D/2-methoxy_dimethyl_1D/2-methoxy_dimethyl_1D_FINAL.vasp


In [6]:
a = np.array([1,0,0])
b = np.array([2,0,1])

print(a-b)

[-1  0 -1]
